# 📰 Klasifikasi Berita dengan Topic Modeling, Stemming, Naïve Bayes, dan SVM
UTS - Analisis Teks & Machine Learning

Notebook ini melakukan klasifikasi berita dengan menambahkan **Stemming Bahasa Indonesia** dan **reduksi fitur (PCA)** setelah ekstraksi topik menggunakan **LDA**.

In [1]:
import pandas as pd
import numpy as np
import re, string
import nltk
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation, PCA
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

nltk.download('stopwords')

# Load dataset
df = pd.read_csv('datasets/Berita.csv')
df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\willy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,No,judul,berita,tanggal,kategori,link
0,1,Airlangga Harap Kenaikan UMP Tingkatkan Daya B...,Menteri Koordinator (Menko) Bidang Perekonomia...,"Minggu, 01 Des 2024 23:40 WIB",Ekonomi,https://www.cnnindonesia.com/ekonomi/202412012...
1,2,PT SIER Beri Penghargaan untuk 50 Tenant Terba...,"Dalam rangka memeriahkan hari jadi ke-50, PT S...","Minggu, 01 Des 2024 20:45 WIB",Ekonomi,https://www.cnnindonesia.com/ekonomi/202412012...
2,3,Prabowo Bakal Bentuk Kementerian Penerimaan Ne...,Wacana Presiden Prabowo Subianto akan membentu...,"Minggu, 01 Des 2024 19:40 WIB",Ekonomi,https://www.cnnindonesia.com/ekonomi/202412011...
3,4,Sinergi Kemenag & BPJS Ketenagakerjaan Lindung...,BPJS Ketenagakerjaan dan Kementerian Agama (Ke...,"Minggu, 01 Des 2024 19:03 WIB",Ekonomi,https://www.cnnindonesia.com/ekonomi/202412011...
4,5,Pemerintah Segera Bentuk Satgas PHK Usai Tetap...,Pemerintah akan segera membentuk Satuan Tugas ...,"Minggu, 01 Des 2024 19:00 WIB",Ekonomi,https://www.cnnindonesia.com/ekonomi/202412011...


## 🔧 Preprocessing dengan Stopword Removal & Stemming

In [2]:
# Inisialisasi stemmer Bahasa Indonesia
factory = StemmerFactory()
stemmer = factory.create_stemmer()
stop_words = set(stopwords.words('indonesian'))

def clean_and_stem(text):
    text = str(text).lower()
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub(r'\d+', '', text)
    tokens = [t for t in text.split() if t not in stop_words]
    stemmed = [stemmer.stem(t) for t in tokens]
    return ' '.join(stemmed)

df = df[['berita', 'kategori']].dropna()
df['clean'] = df['berita'].apply(clean_and_stem)
df.head()

,berita,kategori,clean
0,Menteri Koordinator (Menko) Bidang Perekonomia...,Ekonomi,menteri koordinator menko bidang ekonomi airla...
1,"Dalam rangka memeriahkan hari jadi ke-50, PT S...",Ekonomi,rangka riah pt surabaya industrial estate rung...
2,Wacana Presiden Prabowo Subianto akan membentu...,Ekonomi,wacana presiden prabowo subianto bentuk bentuk...
3,BPJS Ketenagakerjaan dan Kementerian Agama (Ke...,Ekonomi,bpjs ketenagakerjaan menteri agama kemenag lin...
4,Pemerintah akan segera membentuk Satuan Tugas ...,Ekonomi,perintah bentuk satu tugas putus hubung kerja ...


In [10]:
# 11) Export clustered results
from pathlib import Path
out_csv = Path('datasets/preprocessing_berita.csv')
df[['berita','kategori','clean']].to_csv(out_csv, index=False)
print('Saved')

Saved


## 🔠 Ekstraksi Fitur dengan Topic Modeling (LDA)

In [3]:
cv = CountVectorizer(max_features=2000)
X_bow = cv.fit_transform(df['clean'])

# Latent Dirichlet Allocation (LDA)
lda = LatentDirichletAllocation(n_components=6, random_state=42)
topic_features = lda.fit_transform(X_bow)
topic_features.shape

(1500, 6)

## 🔻 Reduksi Fitur dengan PCA

In [4]:
# Reduksi dimensi fitur topik menjadi 2D atau 3D
pca = PCA(n_components=3, random_state=42)
X_reduced = pca.fit_transform(topic_features)
X_reduced.shape

(1500, 3)

## 🔍 Split Data untuk Training dan Testing

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_reduced, df['kategori'], test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((1200, 3), (300, 3))

## 🤖 Klasifikasi Menggunakan Naïve Bayes dan SVM

In [7]:
# Model Naïve Bayes
nb = MultinomialNB()
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)

# Model SVM
svm = SVC(kernel='linear', random_state=42)
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)

# Evaluasi
print('Naïve Bayes Accuracy:', accuracy_score(y_test, y_pred_nb))
print('SVM Accuracy:', accuracy_score(y_test, y_pred_svm))

ValueError: Negative values in data passed to MultinomialNB (input X).

## 📊 Laporan Evaluasi dan Kata Dominan per Topik

In [ ]:
print('--- Naïve Bayes ---')
print(classification_report(y_test, y_pred_nb))

print('\n--- SVM ---')
print(classification_report(y_test, y_pred_svm))

# Kata dominan tiap topik
terms = cv.get_feature_names_out()
for idx, topic in enumerate(lda.components_):
    print(f'\nTopik {idx}:')
    print([terms[i] for i in topic.argsort()[:-10 - 1:-1]])